In [2]:
import pandas as pd
import numpy as np

In [19]:
concatenated_matches_infos_dataframe = pd.read_pickle("../machine_learning/dataframes/data")

percentage_of_win_per_team_after_won_game_1 = concatenated_matches_infos_dataframe[concatenated_matches_infos_dataframe["winner_id"] == concatenated_matches_infos_dataframe["Game 1 winner_id"]].groupby(
    "team_id").size() / concatenated_matches_infos_dataframe.groupby("team_id").size() * 100

percentage_of_win_per_team_after_won_game_1.name = "percentage_of_win_per_team_after_won_game_1"

concatenated_matches_infos_dataframe = pd.merge(concatenated_matches_infos_dataframe, percentage_of_win_per_team_after_won_game_1.to_frame(), left_on="team_id", right_index=True, how="left")

concatenated_matches_infos_dataframe[(concatenated_matches_infos_dataframe["team_id"] == 100) & (concatenated_matches_infos_dataframe["Game 1 winner_id"] == 100)].columns

,match_id,name,tournament_id,number_of_games,winner_id,team_id,opponent_id,tournament_tier,tournament_has_bracket,tournament_name,Game 1 winner_id,Game 2 winner_id,Game 3 winner_id,Game 4 winner_id,Game 5 winner_id,percentage_of_match_win_after_game_1
0,524964,DW vs MMM,2038,1,2796,180,2796,None,0,Regular season,2796,-1,-1,-1,-1,NaN
1,524965,MMM vs LGC,2038,1,2796,2796,181,None,0,Regular season,2796,-1,-1,-1,-1,NaN
2,524968,BMR vs AV,2038,1,1519,1519,179,None,0,Regular season,1519,-1,-1,-1,-1,NaN
3,524970,MMM vs BMR,2038,1,1519,2796,1519,None,0,Regular season,1519,-1,-1,-1,-1,NaN
4,524971,ORD vs DW,2038,1,1521,180,1521,None,0,Regular season,1521,-1,-1,-1,-1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43969,770192,BLX vs JL,10394,1,132476,132476,132656,d,0,Regular Season,132476,-1,-1,-1,-1,NaN
43970,770193,WYL vs SOL,10394,1,130452,133070,130452,d,0,Regular Season,130452,-1,-1,-1,-1,NaN
43971,770194,AKR vs GM,10394,1,133115,133115,133071,d,0,Regular Season,133115,-1,-1,-1,-1,NaN
43972,770195,MDR vs 3DM,10394,1,130451,132474,130451,d,0,Regular Season,130451,-1,-1,-1,-1,NaN


In [70]:
matches_infos_dataframe = pd.read_pickle("../machine_learning/dataframes/data")

team_games_against_opponent = matches_infos_dataframe.groupby(["team_id", "opponent_id"]).size()

team_games_against_opponent.name = "number_of_games_against_opponent"

matches_infos_dataframe = pd.merge(matches_infos_dataframe, team_games_against_opponent, on=["team_id", "opponent_id"], how='left')

df = matches_infos_dataframe[matches_infos_dataframe["team_id"] == matches_infos_dataframe["winner_id"]]

percentage_of_win_against_opponent = df.groupby(["team_id", "opponent_id"]).size() / team_games_against_opponent * 100
percentage_of_win_against_opponent.name = "percentage_of_win_against_opponent"

matches_infos_dataframe = pd.merge(matches_infos_dataframe, percentage_of_win_against_opponent.to_frame(), on=["team_id", "opponent_id"], how='left')

matches_infos_dataframe

KeyError: 'team_id'

In [1]:
matches_infos_dataframe = pd.read_pickle("../machine_learning/dataframes/data")

matches_infos_dataframe

NameError: name 'pd' is not defined

In [53]:
df = pd.read_pickle("../machine_learning/dataframes/final_dataframe")

a = df[(df["team_id"] == df["winner_id"]) & (df["team_id"] == df["Game 1 winner_id"])].groupby("team_id")["team_id"].count()

b = df.groupby("team_id")["team_id"].count()

c = a / b

df = df.merge(c.to_frame(), left_on="team_id", right_index=True, how="left")

df.rename(columns={"team_id_y": "percentage_of_win_after_won_game_1", "team_id_x": "team_id"}, inplace=True)

df['percentage_of_win_against_opponent'] = df.apply(lambda row: df[(df['team_id'] == row['team_id']) & (df['opponent_id'] == row['opponent_id'])]['winner_id'].value_counts(normalize=True).get(row['team_id'], np.nan) if row['team_id'] != row['opponent_id'] else np.nan, axis=1)

a = df[df["team_id"] == df["winner_id"]].groupby(["number_of_games", "team_id"])["team_id"].count()

b = df.groupby(["number_of_games", "team_id"])["team_id"].count()

c = a / b

df = df.merge(c.to_frame(), left_on=["number_of_games", "team_id"], right_index=True, how="left")

df.rename(columns={"team_id_y": "percentage_of_win_after_n_games", "team_id_x": "team_id"}, inplace=True)

df.to_pickle("../machine_learning/dataframes/final_dataframe")

In [51]:
import pickle
from sklearn.impute import SimpleImputer

df = pd.read_pickle("../machine_learning/dataframes/final_dataframe_modified")

X = df[['match_id', 'winner_id', 'tournament_id', 'number_of_games', 'tournament_tier', 'tournament_has_bracket',
        'percentage_of_win_after_won_game_1', 'percentage_of_win_against_opponent', 'percentage_of_win_after_n_games']]

X = X.apply(pd.to_numeric, errors='coerce')

# Imputer les valeurs manquantes avec la moyenne
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

with open("../machine_learning/data/model.pickle", "rb") as file:
    calibrated_model = pickle.load(file)

for i in range(len(df)):
    prediction = calibrated_model.predict_proba([X[i]])[0][1]
    df.at[i, 'win_probability'] = (1 / prediction)

# Afficher le dataframe mis à jour
df

,match_id,name,tournament_id,number_of_games,winner_id,team_id,opponent_id,tournament_tier,tournament_has_bracket,tournament_name,Game 1 winner_id,Game 2 winner_id,Game 3 winner_id,Game 4 winner_id,Game 5 winner_id,percentage_of_win_after_won_game_1,percentage_of_win_against_opponent,percentage_of_win_after_n_games,win_probability
0,524964,DW vs MMM,2038,1,0,180,2796,None,0,Regular season,0,-1,-1,-1,-1,0.531746,0.882353,0.588785,1.378994
1,524965,MMM vs LGC,2038,1,1,2796,181,None,0,Regular season,1,-1,-1,-1,-1,0.352941,0.538462,0.350877,2.136788
2,524968,BMR vs AV,2038,1,1,1519,179,None,0,Regular season,1,-1,-1,-1,-1,0.714286,1.000000,0.730769,1.108809
3,524970,MMM vs BMR,2038,1,0,2796,1519,None,0,Regular season,0,-1,-1,-1,-1,0.352941,0.333333,0.350877,2.341751
4,524971,ORD vs DW,2038,1,0,180,1521,None,0,Regular season,0,-1,-1,-1,-1,0.531746,0.500000,0.588785,2.224107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43969,770192,BLX vs JL,10394,1,1,132476,132656,d,0,Regular Season,1,-1,-1,-1,-1,0.416667,1.000000,0.454545,1.118344
43970,770193,WYL vs SOL,10394,1,0,133070,130452,d,0,Regular Season,0,-1,-1,-1,-1,0.333333,NaN,0.333333,10.988248
43971,770194,AKR vs GM,10394,1,1,133115,133071,d,0,Regular Season,1,-1,-1,-1,-1,0.666667,1.000000,0.666667,1.093664
43972,770195,MDR vs 3DM,10394,1,0,132474,130451,d,0,Regular Season,0,-1,-1,-1,-1,0.363636,NaN,0.363636,11.207944


In [54]:
df.to_pickle("../machine_learning/dataframes/final_dataframe_odds")

In [55]:
df = pd.read_pickle("../machine_learning/dataframes/final_dataframe_odds")
df

,match_id,name,tournament_id,number_of_games,winner_id,team_id,opponent_id,tournament_tier,tournament_has_bracket,tournament_name,Game 1 winner_id,Game 2 winner_id,Game 3 winner_id,Game 4 winner_id,Game 5 winner_id,percentage_of_win_after_won_game_1,percentage_of_win_against_opponent,percentage_of_win_after_n_games,win_probability
0,524964,DW vs MMM,2038,1,0,180,2796,None,0,Regular season,0,-1,-1,-1,-1,0.531746,0.882353,0.588785,1.378994
1,524965,MMM vs LGC,2038,1,1,2796,181,None,0,Regular season,1,-1,-1,-1,-1,0.352941,0.538462,0.350877,2.136788
2,524968,BMR vs AV,2038,1,1,1519,179,None,0,Regular season,1,-1,-1,-1,-1,0.714286,1.000000,0.730769,1.108809
3,524970,MMM vs BMR,2038,1,0,2796,1519,None,0,Regular season,0,-1,-1,-1,-1,0.352941,0.333333,0.350877,2.341751
4,524971,ORD vs DW,2038,1,0,180,1521,None,0,Regular season,0,-1,-1,-1,-1,0.531746,0.500000,0.588785,2.224107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43969,770192,BLX vs JL,10394,1,1,132476,132656,d,0,Regular Season,1,-1,-1,-1,-1,0.416667,1.000000,0.454545,1.118344
43970,770193,WYL vs SOL,10394,1,0,133070,130452,d,0,Regular Season,0,-1,-1,-1,-1,0.333333,NaN,0.333333,10.988248
43971,770194,AKR vs GM,10394,1,1,133115,133071,d,0,Regular Season,1,-1,-1,-1,-1,0.666667,1.000000,0.666667,1.093664
43972,770195,MDR vs 3DM,10394,1,0,132474,130451,d,0,Regular Season,0,-1,-1,-1,-1,0.363636,NaN,0.363636,11.207944
